# Re Run Step Function Executions

### Imports & Creds :

In [1]:
import boto3
import datetime
from dateutil import tz
import sys

# Set up boto3 client for Step Functions
stepfunctions = boto3.client('stepfunctions')

# Constants
STATE_MACHINE_NAME = "RAPISAMStateMachine-1YwcU3XmIcZ5"
SUPPLIER_ID = "SAGREETINGS"
START_TIME_UTC = datetime.datetime(2025, 5, 27, 14, 0, 0, tzinfo=datetime.timezone.utc) # Change this to the start time you want to start from
NOW_UTC = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc)

### Functons :

In [2]:
def get_state_machine_arn(name):
    print(f"[INFO] Looking up state machine ARN for '{name}'...")
    paginator = stepfunctions.get_paginator('list_state_machines')
    for page in paginator.paginate():
        for sm in page['stateMachines']:
            if sm['name'] == name:
                print(f"[INFO] Found ARN: {sm['stateMachineArn']}")
                return sm['stateMachineArn']
    raise ValueError(f"State machine '{name}' not found.")

def list_supplier_executions(state_machine_arn, supplier_id, start_time, end_time):
    print(f"[INFO] Listing executions for supplier '{supplier_id}' between {start_time} and {end_time}...")

    executions = []
    paginator = stepfunctions.get_paginator('list_executions')
    total_scanned = 0
    matched_count = 0

    for page in paginator.paginate(stateMachineArn=state_machine_arn):
        for exe in page['executions']:
            total_scanned += 1

            # Stop if execution is older than the start time (pagination is in DESC order)
            if exe['startDate'] < start_time:
                print("\n[INFO] Reached older executions. Stopping pagination.")
                print(f"[INFO] Total executions scanned: {total_scanned}")
                print(f"[INFO] Matching executions found: {matched_count}")
                return executions

            # Show live scan progress on one line
            sys.stdout.write(f"\r[SCAN] Scanned: {total_scanned} | Matches: {matched_count}")
            sys.stdout.flush()

            if supplier_id in exe['name'] and start_time <= exe['startDate'] <= end_time:
                matched_count += 1
                executions.append(exe)

    print()  # newline after progress bar
    print(f"[INFO] Total executions scanned: {total_scanned}")
    print(f"[INFO] Matching executions found: {matched_count}")
    # return executions
    return print("Scanned: {total_scanned} | Matches: {matched_count}")


def get_execution_input(execution_arn):
    print(f"[INFO] Fetching input for execution ARN: {execution_arn}")
    response = stepfunctions.describe_execution(executionArn=execution_arn)
    return response['input']



### Re Run Executions

In [ ]:
def main():
    print("[INFO] Script started.")
    try:
        state_machine_arn = get_state_machine_arn(STATE_MACHINE_NAME)
        executions = list_supplier_executions(
            state_machine_arn, SUPPLIER_ID, START_TIME_UTC, NOW_UTC
        )

        if not executions:
            print("[INFO] No matching executions found. Exiting.")
            return

    except Exception as e:
        print(f"[ERROR] {e}")

    print("[INFO] Script finished.")

if __name__ == "__main__":
    main()